In [0]:
from .finding_center import find_center

import torch
import torch.optim as optim

import time
import logging

In [0]:
def train(trainloader, classifier, device='cuda', lr=1e-3, weight_decay=1e-6, 
          milestones=[20, 40, 60], c=None, R=None, nu=None, warm_up_n_epochs=None, 
          clf_epochs=100, accumulation_steps=32, objective='one-class', accumulate=False):
    logger = logging.getLogger()

    # Set device for network
    classifier = classifier.to(device)

    # Set optimizer (Adam optimizer for now)
    optimizer = optim.Adam(classifier.parameters(), lr=lr, weight_decay=weight_decay,
                           amsgrad=True)

    # Set learning rate scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1)

    # Initialize hypersphere center c (if c not loaded)
    if c is None:
        logger.info('Initializing center c...')
        c = find_center(trainloader, classifier)
        logger.info('Center c initialized.')

    # Training
    logger.info('Starting training...')
    start_time = time.time()
    classifier.train()
    for epoch in range(clf_epochs):

        scheduler.step()
        if epoch in milestones:
            logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()
        for i, data in enumerate(trainloader):
            inputs, _, _ = data
            inputs = inputs.to(device)

            # Zero the network parameter gradients
            optimizer.zero_grad()

            # Update network parameters via backpropagation: forward + backward + optimize
            outputs = classifier(inputs)
            dist = torch.sum((outputs - c) ** 2, dim=1)
            if objective == 'soft-boundary':
                scores = dist - R ** 2
                loss = R ** 2 + (1 / nu) * torch.mean(torch.max(torch.zeros_like(scores), scores))
            else:
                loss = torch.mean(dist)
            if accumulate:
                loss = loss / accumulation_steps
                loss.backward()
                if (i + 1) % accumulation_steps == 0:
                    optimizer.step()
                    classifier.zero_grad()
            else:
                loss.backward()
                optimizer.step()
            loss_epoch += loss.item()
            n_batches += 1

            # Update hypersphere radius R on mini-batch distances
            if (objective == 'soft-boundary') and (epoch >= warm_up_n_epochs):
                R = torch.tensor(get_radius(dist, nu), device=device)

        # log epoch statistics
        epoch_train_time = time.time() - epoch_start_time
        logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                    .format(epoch + 1, clf_epochs, epoch_train_time, loss_epoch / n_batches))

    train_time = time.time() - start_time
    logger.info('Training time: %.3f' % train_time)

    logger.info('Finished training.')

    return classifier